In [26]:
import pandas as pd
import numpy as np
import os
from utilities.utils import ErrorFunctions, SSPModelForCalibration
from utilities.diff_reports import DiffReportUtils

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Define paths
curr_dir = os.getcwd()
misc_dir = os.path.join(curr_dir, 'misc')
sectoral_report_dir = os.path.join(misc_dir, 'sectoral_reports')
dummy_files_dir = os.path.join(misc_dir, 'dummy')
sectoral_report_mapping_dir = os.path.join(misc_dir, 'sectoral_report_mapping')


In [29]:
# Initialize the diff report object
dru = DiffReportUtils('UGA', os.path.join(sectoral_report_mapping_dir, 'sisepuede_edgar_active_crosswalk.csv'), sectoral_report_dir, energy_model_flag=False, comparison_year=2022, sim_init_year=2022)

In [30]:
# Load the edgar emissions database for the desired region
edgar_emissions = dru.edgar_emission_db_etl(os.path.join(sectoral_report_mapping_dir, 'CSC-GHG_emissions-April2024_to_calibrate.csv'))
edgar_emissions

,iso_alpha_3,edgar_class,edgar_emission,year
0,UGA,AG - Livestock:CH4,19.944488,2022
1,UGA,AG - Livestock:N2O,0.299059,2022
2,UGA,AG - Crops:CH4,0.576887,2022
3,UGA,AG - Crops:N2O,8.092843,2022
4,UGA,EN - Building:CH4,2.554441,2022
5,UGA,EN - Building:CO2,0.611469,2022
6,UGA,EN - Building:N2O,0.323356,2022
7,UGA,EN - Electricity/Heat:CH4,0.003233,2022
8,UGA,EN - Electricity/Heat:CO2,0.056306,2022
9,UGA,EN - Electricity/Heat:N2O,0.004118,2022


## Test Diff Report Class Methods

In [12]:
ssp_out = pd.read_csv(os.path.join(dummy_files_dir, 'ssp_uganda_output_dummy.csv'))

In [13]:
ssp_out.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,1.110674e+06,2.470537e+06,512618.657191,1.336131e+06,1.724155e+06,940987.489485,1.244761e+06,830632.082001,571949.517471,...,9.851960e+06,3.660909e+06,717057.782374,1.223273e+06,305174.162968,448689.809992,1.088927e+06,5.904929e+07,2.194701e+06,1.187277e+06
1,1,1.099568e+06,2.445834e+06,507493.032449,1.322771e+06,1.706915e+06,931578.645912,1.232315e+06,822326.671554,566230.649153,...,9.536923e+06,3.706026e+06,667169.430627,1.213181e+06,302122.755810,455151.199769,1.105636e+06,5.896535e+07,2.227650e+06,1.195600e+06
2,2,1.108967e+06,2.466741e+06,511831.054023,1.334078e+06,1.721506e+06,939541.727186,1.242849e+06,829355.873165,571070.756532,...,9.588445e+06,3.678309e+06,724448.535785,1.242594e+06,304705.283940,461647.924279,1.147618e+06,5.931858e+07,2.564269e+06,1.186876e+06
3,3,1.097881e+06,2.442080e+06,506714.112877,1.320741e+06,1.704295e+06,930148.823640,1.230424e+06,821064.533360,565361.576856,...,9.473999e+06,3.637027e+06,692803.554751,1.773441e+06,301659.046334,454988.571853,1.156682e+06,5.836201e+07,2.735081e+06,1.177157e+06
4,4,1.107827e+06,2.464205e+06,511304.810668,1.332707e+06,1.719736e+06,938575.729546,1.241571e+06,828503.164027,570483.605385,...,9.621189e+06,3.735378e+06,429980.347008,1.897525e+06,304391.998669,456280.825701,1.170637e+06,5.961326e+07,2.366263e+06,1.194025e+06


In [14]:
ssp_edgar_df = dru.load_ssp_edgar_cw()
ssp_edgar_df.tail(20)

,subsector,gas,edgar_class,edgar_subsector,edgar_sector,ignore,note,need_better_information_on_what_is_contained,vars
23,ippu,n2o,IN - Industrial Processes:N2O,IN - Industrial Processes,Industrial Processes,NaN,NaN,NaN,emission_co2e_n2o_ippu_production_chemicals:em...
24,ippu,hfcs,IN - Industrial Processes:HFC,IN - Industrial Processes,Industrial Processes,NaN,NOTE: can pick this set (HFCs aggregate) or sp...,NaN,emission_co2e_hfcs_ippu_product_use_product_us...
25,ippu,other_fcs,IN - Industrial Processes:HFC,IN - Industrial Processes,Industrial Processes,NaN,NOTE: these aren't HFCs,NaN,NaN
26,ippu,pfcs,IN - Industrial Processes:PFC,IN - Industrial Processes,Industrial Processes,NaN,NaN,NaN,emission_co2e_pfcs_ippu_product_use_product_us...
27,ippu,sf6,IN - Industrial Processes:SF6,IN - Industrial Processes,Industrial Processes,NaN,NaN,NaN,emission_co2e_sf6_ippu_production_chemicals:em...
28,waso,co2,Waste - Solid Waste:CO2,Waste - Solid Waste,Waste,NaN,NaN,NaN,emission_co2e_co2_waso_incineration
29,waso,ch4,Waste - Solid Waste:CH4,Waste - Solid Waste,Waste,NaN,NaN,NaN,emission_co2e_ch4_waso_biogas_food:emission_co...
30,waso,n2o,Waste - Solid Waste:N2O,Waste - Solid Waste,Waste,NaN,NaN,NaN,emission_co2e_n2o_waso_compost_food:emission_c...
31,trww,ch4,Waste - Wastewater Treatment:CH4,Waste - Wastewater Treatment,Waste,NaN,NaN,NaN,emission_co2e_ch4_trww_treated_advanced_aerobi...
32,trww,n2o,Waste - Wastewater Treatment:N2O,Waste - Wastewater Treatment,Waste,NaN,NaN,NaN,emission_co2e_n2o_trww_treated_advanced_aerobi...


In [15]:
# ssp_edgar_df[ssp_edgar_df.ignore == 1]

In [16]:
ssp_emissions_report, model_failed_flag = dru.generate_ssp_emissions_report(ssp_out)
ssp_emissions_report

,subsector,gas,edgar_class,ssp_emission
0,lvst,ch4,AG - Livestock:CH4,59.862185
1,lsmm,ch4,AG - Livestock:CH4,2.511765
2,lsmm,n2o,AG - Livestock:N2O,6.355724
3,agrc,co2,AG - Crops:CO2,0.864601
4,agrc,ch4,AG - Crops:CH4,1.771977
5,agrc,n2o,AG - Crops:N2O,0.565280
6,scoe,co2,EN - Building:CO2,7.702788
7,scoe,ch4,EN - Building:CH4,0.033125
8,scoe,n2o,EN - Building:N2O,0.058250
9,entc,co2,EN - Electricity/Heat:CO2,NaN


In [17]:
ssp_emissions_report = dru.group_ssp_emissions_report_vars(ssp_emissions_report)
ssp_emissions_report

,subsector,edgar_class,ssp_emission
0,agrc,AG - Crops:CH4,1.771977
1,agrc,AG - Crops:CO2,0.864601
2,agrc,AG - Crops:N2O,0.565280
3,ccsq,CCSQ:CH4,0.000000
4,ccsq,CCSQ:CO2,0.000000
5,ccsq,CCSQ:N2O,0.000000
6,entc,EN - Electricity/Heat:CH4,0.000000
7,entc,EN - Electricity/Heat:CO2,0.000000
8,entc,EN - Electricity/Heat:N2O,0.000000
9,fgtv,EN - Fugitive Emissions:CH4,0.000000


In [18]:
# check energy subsector
energy_subsectors = ['trns', 'entc', 'fgtv', 'scoe', 'inen', 'ccsq']
ssp_emissions_report[ssp_emissions_report.subsector.isin(energy_subsectors)]

,subsector,edgar_class,ssp_emission
3,ccsq,CCSQ:CH4,0.000000
4,ccsq,CCSQ:CO2,0.000000
5,ccsq,CCSQ:N2O,0.000000
6,entc,EN - Electricity/Heat:CH4,0.000000
7,entc,EN - Electricity/Heat:CO2,0.000000
8,entc,EN - Electricity/Heat:N2O,0.000000
9,fgtv,EN - Fugitive Emissions:CH4,0.000000
10,fgtv,EN - Fugitive Emissions:CO2,0.000000
11,fgtv,EN - Fugitive Emissions:N2O,0.000000
14,inen,EN - Manufacturing/Construction:CH4,0.108583


In [19]:
model_failed_flag

False

In [20]:
edgar_emissions.sort_values(by=['edgar_class'], ascending=True)

,iso_alpha_3,edgar_class,edgar_emission,year
2,UGA,AG - Crops:CH4,0.576887,2022
3,UGA,AG - Crops:N2O,8.092843,2022
0,UGA,AG - Livestock:CH4,19.944488,2022
1,UGA,AG - Livestock:N2O,0.299059,2022
4,UGA,EN - Building:CH4,2.554441,2022
5,UGA,EN - Building:CO2,0.611469,2022
6,UGA,EN - Building:N2O,0.323356,2022
7,UGA,EN - Electricity/Heat:CH4,0.003233,2022
8,UGA,EN - Electricity/Heat:CO2,0.056306,2022
9,UGA,EN - Electricity/Heat:N2O,0.004118,2022


In [21]:
df_report = dru.merge_ssp_with_edgar(ssp_emissions_report, edgar_emissions)
df_report


,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year
0,agrc,AG - Crops:CH4,1.771977,UGA,0.576887,2022
1,agrc,AG - Crops:CO2,0.864601,NaN,NaN,2022
2,agrc,AG - Crops:N2O,0.565280,UGA,8.092843,2022
3,ccsq,CCSQ:CH4,0.000000,NaN,NaN,2022
4,ccsq,CCSQ:CO2,0.000000,NaN,NaN,2022
5,ccsq,CCSQ:N2O,0.000000,NaN,NaN,2022
6,entc,EN - Electricity/Heat:CH4,0.000000,UGA,0.003233,2022
7,entc,EN - Electricity/Heat:CO2,0.000000,UGA,0.056306,2022
8,entc,EN - Electricity/Heat:N2O,0.000000,UGA,0.004118,2022
9,fgtv,EN - Fugitive Emissions:CH4,0.000000,UGA,7.845444,2022


In [39]:
dru.group_trns_scoe_inen(df_report)

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year
0,inen,EN - Manufacturing/Construction,4.069753,UGA,1.289418,2015
1,scoe,EN - Building,7.547604,UGA,2.897600,2015
2,trns,EN - Transportation,3.826507,UGA,2.794684,2015
3,agrc,AG - Crops:CH4,0.214505,UGA,0.422303,2015
4,agrc,AG - Crops:CO2,0.100414,NaN,NaN,2015
5,agrc,AG - Crops:N2O,0.247102,UGA,7.141899,2015
6,ccsq,CCSQ:CH4,0.000000,NaN,NaN,2015
7,ccsq,CCSQ:CO2,0.000000,NaN,NaN,2015
8,ccsq,CCSQ:N2O,0.000000,NaN,NaN,2015
9,entc,EN - Electricity/Heat:CH4,0.000000,UGA,0.011616,2015


In [40]:
# get duplicated edgar_class
df_report[df_report.duplicated(subset=['edgar_class'], keep=False)]

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year


In [41]:
reports_dict = dru.run_report_generator(edgar_emissions, ssp_out)

sectoral_emission_report = reports_dict['sectoral_emission_report']
subsector_emission_report = reports_dict['subsector_emission_report']

In [42]:
sectoral_emission_report[sectoral_emission_report['subsector'].isin(energy_subsectors)]

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
0,inen,EN - Manufacturing/Construction,4.069753,UGA,1.289418,2015,1.289419,2.156270,7.730258,1.289419,0.013805,0.828297
1,scoe,EN - Building,7.547604,UGA,2.897600,2015,2.897601,1.604777,21.622534,2.897601,0.031024,1.360361
2,trns,EN - Transportation,3.826507,UGA,2.794684,2015,2.794685,0.369208,1.064656,2.794685,0.029922,1.333601


In [43]:
sectoral_emission_report

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
0,inen,EN - Manufacturing/Construction,4.069753,UGA,1.289418,2015,1.289419,2.156270e+00,7.730258,1.289419,0.013805,0.828297
1,scoe,EN - Building,7.547604,UGA,2.897600,2015,2.897601,1.604777e+00,21.622534,2.897601,0.031024,1.360361
2,trns,EN - Transportation,3.826507,UGA,2.794684,2015,2.794685,3.692085e-01,1.064656,2.794685,0.029922,1.333601
3,agrc,AG - Crops:CH4,0.214505,UGA,0.422303,2015,0.422304,-4.920602e-01,0.043180,0.422304,0.004521,0.352278
5,agrc,AG - Crops:N2O,0.247102,UGA,7.141899,2015,7.141900,-9.654011e-01,47.538233,7.141900,0.076466,2.097023
16,frst,LULUCF - Forest Land:CO2,-1.100202,UGA,40.000000,2015,40.000001,-1.027505e+00,1689.226691,40.000001,0.428267,3.713572
18,ippu,IN - Industrial Processes:CO2,7.361243,UGA,1.011268,2015,1.011269,6.279216e+00,40.322170,1.011269,0.010827,0.698765
20,ippu,IN - Industrial Processes:N2O,2.553071,UGA,0.484917,2015,0.484918,4.264952e+00,4.277257,0.484918,0.005192,0.395359
24,lndu,LULUCF - Deforestation:CO2,0.242741,UGA,13.000000,2015,13.000001,-9.813276e-01,162.747690,13.000001,0.139187,2.639057
25,lndu,LULUCF - Other Land:CO2,0.025623,UGA,0.000000,2015,0.000001,2.562200e+04,0.000656,0.000001,0.000000,0.000000


In [44]:
sectoral_emission_report.tail(10)

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
18,ippu,IN - Industrial Processes:CO2,7.361243,UGA,1.011268,2015,1.011269,6.279216e+00,40.322170,1.011269,0.010827,0.698765
20,ippu,IN - Industrial Processes:N2O,2.553071,UGA,0.484917,2015,0.484918,4.264952e+00,4.277257,0.484918,0.005192,0.395359
24,lndu,LULUCF - Deforestation:CO2,0.242741,UGA,13.000000,2015,13.000001,-9.813276e-01,162.747690,13.000001,0.139187,2.639057
25,lndu,LULUCF - Other Land:CO2,0.025623,UGA,0.000000,2015,0.000001,2.562200e+04,0.000656,0.000001,0.000000,0.000000
26,lsmm,AG - Livestock:N2O,0.165996,UGA,0.269423,2015,0.269424,-3.838859e-01,0.010697,0.269424,0.002885,0.238562
27,lvst-lsmm,AG - Livestock:CH4,1.979247,UGA,17.565323,2015,17.565324,-8.873208e-01,242.925805,17.565324,0.188066,2.921295
28,soil,LULUCF - Organic Soil:CO2,1.216540,UGA,0.000000,2015,0.000001,1.216539e+06,1.479966,0.000001,0.000000,0.000000
30,trww,Waste - Wastewater Treatment:CH4,0.394961,UGA,5.747256,2015,5.747257,-9.312784e-01,28.647074,5.747257,0.061534,1.909136
31,trww,Waste - Wastewater Treatment:N2O,0.270326,UGA,0.365269,2015,0.365270,-2.599301e-01,0.009015,0.365270,0.003911,0.311352
32,waso,Waste - Solid Waste:CH4,0.832718,UGA,0.410339,2015,0.410340,1.029338e+00,0.178403,0.410340,0.004393,0.343830


In [28]:
sectoral_emission_report[sectoral_emission_report['subsector'] == 'ippu']

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
17,ippu,IN - Industrial Processes:CH4,0.034023,HRV,0.001673,2015,0.001674,19.323149,1.046427e-03,0.001674,0.000061,0.001672
18,ippu,IN - Industrial Processes:CO2,7.361243,HRV,2.415891,2015,2.415892,2.047009,2.445650e+01,2.415892,0.087991,1.228438
19,ippu,IN - Industrial Processes:HFC,1.219243,HRV,0.514149,2015,0.514150,1.371375,4.971556e-01,0.514150,0.018726,0.414854
20,ippu,IN - Industrial Processes:N2O,2.553071,HRV,0.946129,2015,0.946130,1.698437,2.582262e+00,0.946130,0.034460,0.665842
21,ippu,IN - Industrial Processes:PFC,0.000008,HRV,0.000000,2015,0.000001,7.248265,5.253735e-11,0.000001,0.000000,0.000000
22,ippu,IN - Industrial Processes:SF6,0.000000,HRV,0.005376,2015,0.005377,-1.000000,2.891163e-05,0.005377,0.000196,0.005362


In [29]:
subsector_emission_report

,subsector,ssp_emission,edgar_emission_epsilon,rel_error,squared_diff,year
0,agrc,1.778561,1.134142,0.568199,0.415276,2015
1,frst,-1.247014,-6.547103,-0.809532,28.090943,2015
2,inen,4.069753,2.157341,0.886467,3.657318,2015
3,ippu,11.167588,3.883223,1.875855,53.061963,2015
4,lndu,0.268364,0.881078,-0.695415,0.375419,2015
5,lsmm,0.165996,0.110194,0.506401,0.003114,2015
6,lvst-lsmm,1.979247,1.682917,0.176081,0.087812,2015
7,scoe,7.547604,3.247698,1.323986,18.489194,2015
8,soil,0.110122,0.092260,0.193601,0.000319,2015
9,trns,3.826507,5.958180,-0.357772,4.544028,2015


### Test new error function

In [125]:
input_df = pd.read_csv('misc/dummy/ssp_output_dummy.csv')
input_df.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,128992.475401,286925.570284,59534.988647,155176.845408,200241.524314,109285.291660,144565.284932,96468.731604,66425.612333,...,860714.235864,5.781814e+06,65829.816853,279108.581486,50211.974800,175242.943363,0.0,0.0,2.050396e+06,725849.288265
1,1,128305.935425,285398.458926,59218.124042,154350.943684,199175.773688,108703.639735,143795.861390,95955.293587,66072.073584,...,857478.660027,5.906717e+06,52166.876709,331312.854435,70450.376552,215192.338859,2736.0,2449.0,2.713049e+06,699533.070417
2,2,127621.990578,283877.120067,58902.457190,153528.163879,198114.051612,108124.186462,143029.346278,95443.796372,65719.871216,...,660924.245615,5.951148e+06,38967.956801,296921.771405,82224.830575,210243.344677,2736.0,2449.0,2.331104e+06,761148.281366
3,3,126940.639041,282361.549661,58587.987250,152708.503803,197056.355263,107546.930298,142265.737557,94934.238599,65369.004292,...,957822.768981,5.919376e+06,17239.772927,378730.362142,93529.211867,213539.618942,2736.0,2449.0,2.111415e+06,729155.981592
4,4,126261.879236,280851.744198,58274.713494,151891.961560,196002.682190,106971.869908,141505.033460,94426.619087,65019.472000,...,892950.504956,5.887725e+06,21001.252360,359927.486275,85408.876964,264501.812874,2736.0,2449.0,2.234268e+06,737682.556359


In [126]:
dru.run_report_generator(edgar_emissions, input_df)

Index(['subsector', 'gas', 'edgar_class', 'edgar_subsector', 'edgar_sector',
       'ignore', 'note', 'need_better_information_on_what_is_contained',
       'vars'],
      dtype='object')
Missing variables for lvst False
Missing variables for lsmm False
Missing variables for lsmm False
Missing variables for agrc False
Missing variables for agrc False
Missing variables for agrc False
Missing variables for scoe False
Missing variables for scoe False
Missing variables for scoe False
Missing variables for entc True
Missing variables for entc True
Missing variables for entc True
Missing variables for fgtv True
Missing variables for fgtv True
Missing variables for fgtv True
Missing variables for inen False
Missing variables for inen False
Missing variables for inen False
Missing variables for trns False
Missing variables for trns False
Missing variables for trns False
Missing variables for ippu False
Missing variables for ippu False
Missing variables for ippu False
Missing variables for ippu

In [127]:
df = dru.sectoral_emission_report.copy()
df.head()

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
0,inen,EN - Manufacturing/Construction,2.808345,HRV,2.157340,2015,2.157341,0.301762,0.423806,2.157341,0.078574,1.149730
1,scoe,EN - Building,4.064501,HRV,3.247697,2015,3.247698,0.251502,0.667166,3.247698,0.118287,1.446377
2,trns,EN - Transportation,6.009059,HRV,5.958179,2015,5.958180,0.008539,0.002589,5.958180,0.217008,1.939918
3,agrc,AG - Crops:CH4,0.290094,HRV,0.000227,2015,0.000228,1273.348921,0.084023,0.000228,0.000008,0.000227
4,agrc,AG - Crops:CO2,0.065572,HRV,0.095857,2015,0.095858,-0.315950,0.000917,0.095858,0.003491,0.091537


In [128]:
df.to_csv('misc/dummy/sectoral_emission_report_dummy.csv', index=False)

In [129]:
df.sort_values(by=['norm_weight'], ascending=False, inplace=True)

In [130]:
df

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
16,frst,LULUCF - Forest Land:CO2,-1.650303,HRV,-6.547104,2015,-6.547103,-0.747934,2.397865e+01,6.547105,0.238458,2.021164
2,trns,EN - Transportation,6.009059,HRV,5.958179,2015,5.958180,0.008539,2.588703e-03,5.958180,0.217008,1.939918
1,scoe,EN - Building,4.064501,HRV,3.247697,2015,3.247698,0.251502,6.671663e-01,3.247698,0.118287,1.446377
18,ippu,IN - Industrial Processes:CO2,3.669455,HRV,2.415891,2015,2.415892,0.518883,1.571422e+00,2.415892,0.087991,1.228438
0,inen,EN - Manufacturing/Construction,2.808345,HRV,2.157340,2015,2.157341,0.301762,4.238060e-01,2.157341,0.078574,1.149730
27,lvst-lsmm,AG - Livestock:CH4,1.658163,HRV,1.682916,2015,1.682917,-0.014709,6.127554e-04,1.682917,0.061295,0.986904
32,waso,Waste - Solid Waste:CH4,0.829621,HRV,1.255624,2015,1.255625,-0.339276,1.814796e-01,1.255625,0.045732,0.813427
5,agrc,AG - Crops:N2O,0.377462,HRV,1.038055,2015,1.038056,-0.636377,4.363853e-01,1.038056,0.037808,0.711996
20,ippu,IN - Industrial Processes:N2O,1.276536,HRV,0.946129,2015,0.946130,0.349219,1.091682e-01,0.946130,0.034460,0.665842
25,lndu,LULUCF - Other Land:CO2,0.014875,HRV,0.820237,2015,0.820238,-0.981865,6.486093e-01,0.820238,0.029875,0.598967


In [131]:
df['norm_weight'] * 100

16    23.845818
2     21.700839
1     11.828741
18     8.799141
0      7.857450
27     6.129504
32     4.573227
5      3.780798
20     3.445983
25     2.987461
19     1.872631
30     1.608924
26     0.401343
4      0.349130
28     0.336027
31     0.223730
24     0.221589
22     0.019580
34     0.011051
17     0.006094
3      0.000825
33     0.000112
21     0.000000
Name: norm_weight, dtype: float64

In [132]:
ef = ErrorFunctions()
ef.calculate_error(error_type='wmape', dataframe=df)

1.9768234816325139

In [133]:
np.sum(df['norm_weight'] * df['rel_error'].abs()) * 100

45.46694007754782